In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [5]:
df=pd.read_csv('glass.csv')

# 전처리

In [19]:
x_data=df.iloc[:,:-1]
y_data=df[['Type']]

# 표준화

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(x_data)

# 테스트 트레인 스플릿

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(scaled,y_data,test_size=0.2,random_state=121)

# 검증 함수

In [83]:
from sklearn.metrics import confusion_matrix,precision_score, recall_score,f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [37]:
grid_cv.get_params()

{'cv': 2,
 'error_score': nan,
 'estimator': RandomForestClassifier(random_state=0),
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': 0,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'n_jobs': -1,
 'param_grid': {'max_depth': [6, 8, 10, 12, 13, 14],
  'min_samples_leaf': [3, 4, 5, 6],
  'min_samples_split': [1, 2, 3, 4, 5, 6],
  'n_estimators': [90, 100, 110, 120]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

# 결정트리 및 개별모델

## 의사결정나무

In [69]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train , y_train)
dt_pred=dt_clf.predict(X_test)
accuracy_score(y_test,dt_pred)

0.6744186046511628

## 로지스틱 회귀

In [73]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=156)
lr.fit(X_train , y_train)
lr_pred=lr.predict(X_test)

## knn근접
    - 어떤 데이터가 주어지면 그 주변(이웃)의 데이터를 살펴본 뒤 더 많은 데이터가 포함되어 있는 범주로 분류하는 방식
    - K: 참고할 이웃의 수. 기본값은 5

In [71]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=8)

classifiers = [lr, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))


LogisticRegression 정확도: 0.4884
KNeighborsClassifier 정확도: 0.5581


#앙상블

## VotingClassifier

In [72]:
from sklearn.ensemble import VotingClassifier
vo_clf = VotingClassifier( estimators=[('LR',lr),('KNN',knn_clf)] , voting='soft' )

vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

Voting 분류기 정확도: 0.5814


## SVM(보팅)
  - 보팅: 서로 다른 알고리즘 여러개 결합. 선형회귀, K 최근접 이웃, SVM 알고리즘 등 다양한 알고리즘을 결합하여 만든다.
    - 하드 보팅: 다수 분류기가 결정한 예측을 선택
    - 소프트 보팅: 분류기들의 레이블 결정 확률의 평균을 내어 이 중 가장 큰 확률의 레이블 선택
    - svm
      - 결정 경계(Decision Boundary), 즉 분류를 위한 기준 선을 정의하는 모델
최적의 경계를 찾는 것이 목적
      - 분류하는 데이터들 사이의 마진이 최대가 되는 선이 좋다    

In [68]:
from sklearn import svm
clf = svm.SVC()

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)

print('svm 정확도:', acc)

svm 정확도: 0.6511627906976745


## RandomForestClassifier(배깅)
  - 배깅: 같은 알고리즘 여러개 결합. 랜덤 포레스트 알고리즘이 이에 속함
      - 랜덤 포레스트: 대표적 배깅 알고리즘으로 빠르고 예측 성능이 좋다 결정트리 여러개로 구성.
      - 각 분류기는 서로 다른 데이터셋을 학습하지만 일부 데이터는 중첩되어 있다. 이를 부트스트래핑이라 함.

In [33]:
# randomforestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf_clf=RandomForestClassifier(random_state=0)
rf_clf.fit(X_train,y_train)
pred=rf_clf.predict(X_test)
accuracy_score(y_test,pred)

0.6511627906976745

## GridSearchCV(최적파라미터찾기)

In [40]:

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[90],
    'max_depth' : [6], 
    'min_samples_leaf' : [6,7,8],
    'min_samples_split' : [6,7,8]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 6, 'n_estimators': 90}
최고 예측 정확도: 0.6667


In [67]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GridSearchCV 정확도: {0:.4f}'.format(gb_accuracy))

GridSearchCV 정확도: 0.6047


## GBClassifier(부스팅)
    - 여러 약한 학습기가 순차적으로 느슨하게 분류하고 오분류된 데이터에 대해 가중치를 부여하여, 예측 결정 기준을 모두 결합해 예측을 수행
    - 가중치 업데이트를 경사 하강법을 이용

In [66]:

from sklearn.ensemble import GradientBoostingClassifier
import time

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
print(gb_pred)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBClassifier 정확도: {0:.4f}'.format(gb_accuracy))
print("GBClassifier 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

[7 2 2 2 1 2 1 1 7 2 3 1 7 2 2 1 2 5 1 1 1 1 1 1 1 1 1 1 1 2 1 1 3 1 7 1 2
 3 1 3 2 1 2]
GBClassifier 정확도: 0.6279
GBClassifier 수행 시간: 0.7 초 


## xgboost(0,1로 해야해서 불가능)

    - GBM에 기반하고 있고, 단점인 느린 수행 시간 및 과적합 등의 문제를 해결했으며 특히 병렬 학습이 가능해 기존 GBM보다 빠른 학습이 가능하다.
    - 뛰어난 예측 성능
    - GBM 대비 빠른 수행 시간
    - 과적합 규제
    - 나무 가지치기: 의미 없는 분할 제거
    - 결손값 자체 처리
    - 자체 내장된 교차 검증

In [85]:
import xgboost as xgb
from xgboost import plot_importance

dtrain = xgb.DMatrix(data=X_train , label=y_train)
dtest = xgb.DMatrix(data=X_test , label=y_test)
params = { 'max_depth':3,
           'eta': 0.1,
           'objective':'binary:logistic',
           'eval_metric':'logloss'
        }
num_rounds = 400

wlist = [(dtrain,'train'),(dtest,'eval') ]

xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds ,early_stopping_rounds=100, evals=wlist )
pred_probs = xgb_model.predict(dtest)
print('predict( ) 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10],3))

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds에 저장 
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
print('예측값 10개만 표시:',preds[:10])

XGBoostError: ignored

## LightGBM
    - XGBoost보다 학습 시간이 훨씬 적고 메모리 사용량도 작다. 하지만 적은 데이터 세트에 적용할 경우 과적합이 쉽게 발생하는 단점이 있다.
    - 앞서 살펴본 알고리즘들은 균형 트리를 생성하느라 시간 및 비용이 많이 드는 단점을 극복하기 위해 리프 중심 트리 분할 방법을 사용한다. 이 방법은 균형을 맞추지 않고 계속 분할을 진행하여 트리 깊이가 깊어지고 비대칭 트리가 생성된다.
    - 더 빠른 학습과 예측 수행 시간
    - 더 작은 메모리 사용량
    - 카테고리형 피처의 자동 변환과 최적 분할

In [92]:
from lightgbm import LGBMClassifier


lgbm_wrapper = LGBMClassifier(n_estimators=400)


evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]
accuracy_score(y_test,lgbm_preds)

[1]	valid_0's multi_logloss: 1.59588	valid_0's multi_logloss: 1.59588
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 1.54822	valid_0's multi_logloss: 1.54822
[3]	valid_0's multi_logloss: 1.50823	valid_0's multi_logloss: 1.50823
[4]	valid_0's multi_logloss: 1.46851	valid_0's multi_logloss: 1.46851
[5]	valid_0's multi_logloss: 1.42718	valid_0's multi_logloss: 1.42718
[6]	valid_0's multi_logloss: 1.39311	valid_0's multi_logloss: 1.39311
[7]	valid_0's multi_logloss: 1.36452	valid_0's multi_logloss: 1.36452
[8]	valid_0's multi_logloss: 1.34061	valid_0's multi_logloss: 1.34061
[9]	valid_0's multi_logloss: 1.30484	valid_0's multi_logloss: 1.30484
[10]	valid_0's multi_logloss: 1.29024	valid_0's multi_logloss: 1.29024
[11]	valid_0's multi_logloss: 1.27821	valid_0's multi_logloss: 1.27821
[12]	valid_0's multi_logloss: 1.26077	valid_0's multi_logloss: 1.26077
[13]	valid_0's multi_logloss: 1.23598	valid_0's multi_logloss: 1.23598
[14]	valid_0's multi_lo

0.627906976744186

0.627906976744186